# Explore Merging Inputs into Xarray Datasets

# Imports
Using the environment that you set up in the example notebook. Run the following. 


In [30]:
import sys
import os
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

# Demo Merging Inputs into Single DataSet
Variations on Paul's refatoring branch: https://github.com/EcohydrologyTeam/ClearWater-modules/blob/bd0c52267f4000303c1d593f7975ad8fc5423788/src/clearwater_modules_v2/run_dev.py 

In [2]:
Path.cwd()

PosixPath('/Users/aaufdenkampe/Documents/Python/ClearWater-modules/examples/dev_sandbox')

## Read CSVs & Process into Dataframes

In [3]:
model_name = 'sumwere_creek_coarse_p48'

In [4]:
# Path to the data directory
user = "anthony"

if user == "paul":
    DATA_ROOT = Path(
        r"C:\Users\ptomasula\Repositories\ClearWater-modules\data_temp"
    ) / model_name
else:
    DATA_ROOT = (
        Path.cwd().parent.parent.parent 
        / "Clearwater-riverine/examples/data_temp" 
        / model_name
    )
DATA_ROOT.exists()

True

In [5]:
def process_csv_inputs(
    csv_path: os.PathLike,
    index_col: int | str = 0,
) -> pd.DataFrame: 
    df = pd.read_csv(csv_path, index_col=index_col, parse_dates=True)
    df.dropna(axis="index", how="all", inplace=True)
    frequency = pd.infer_freq(df.index)
    df.index.freq = frequency
    df.index.rename(f"time_{frequency}", inplace=True)
    return df


In [6]:
### Air Temperature
### Measured at 5 min intervals, such as from a weather station
air_temperature_df = process_csv_inputs(
    DATA_ROOT / "cwr_boundary_conditions_TairC_p28.csv",
)
air_temperature_df.info()
air_temperature_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1440 entries, 2022-05-12 00:00:00 to 2022-05-16 23:55:00
Freq: 5min
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TairC   1440 non-null   float64
dtypes: float64(1)
memory usage: 22.5 KB


,TairC
time_5min,
2022-05-12 00:00:00,66.0
2022-05-12 00:05:00,66.0
2022-05-12 00:10:00,66.0
2022-05-12 00:15:00,66.0
2022-05-12 00:20:00,65.0
...,...
2022-05-16 23:35:00,72.0
2022-05-16 23:40:00,72.0
2022-05-16 23:45:00,71.0


In [7]:
### Solar radiation
# Hourly estimates, such as from a climate reanalysis dataset 
# (i.e. ERA5, NLDAS, or CONUS404)
solar_df = process_csv_inputs(
    DATA_ROOT / "cwr_boundary_conditions_q_Solar_p28.csv",
)
solar_df.info()
solar_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 168 entries, 2022-05-10 17:00:00 to 2022-05-17 16:00:00
Freq: h
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   q_Solar  168 non-null    float64
dtypes: float64(1)
memory usage: 2.6 KB


,q_Solar
time_h,
2022-05-10 17:00:00,447.101675
2022-05-10 18:00:00,239.106842
2022-05-10 19:00:00,60.404514
2022-05-10 20:00:00,0.090223
2022-05-10 21:00:00,0.000000
...,...
2022-05-17 12:00:00,927.736352
2022-05-17 13:00:00,948.337344
2022-05-17 14:00:00,905.315856


In [8]:
# Add other required hourly meteorological boundary conditions
met_hourly_df = solar_df.copy()
met_hourly_df["cloudiness"] = 0.1
met_hourly_df["wind_speed"] = 3.0
met_hourly_df["atmospheric_pressure"] = 1013.0
met_hourly_df["atmospheric_vapor_pressure"] = (
    met_hourly_df["atmospheric_pressure"]
    - np.random.normal(0, 1, size=len(met_hourly_df))
)
met_hourly_df.info()
met_hourly_df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 168 entries, 2022-05-10 17:00:00 to 2022-05-17 16:00:00
Freq: h
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   q_Solar                     168 non-null    float64
 1   cloudiness                  168 non-null    float64
 2   wind_speed                  168 non-null    float64
 3   atmospheric_pressure        168 non-null    float64
 4   atmospheric_vapor_pressure  168 non-null    float64
dtypes: float64(5)
memory usage: 7.9 KB


,q_Solar,cloudiness,wind_speed,atmospheric_pressure,atmospheric_vapor_pressure
time_h,,,,,
2022-05-10 17:00:00,447.101675,0.1,3.0,1013.0,1013.432787
2022-05-10 18:00:00,239.106842,0.1,3.0,1013.0,1013.727130
2022-05-10 19:00:00,60.404514,0.1,3.0,1013.0,1013.867946
2022-05-10 20:00:00,0.090223,0.1,3.0,1013.0,1012.757997
2022-05-10 21:00:00,0.000000,0.1,3.0,1013.0,1012.491687
...,...,...,...,...,...
2022-05-17 12:00:00,927.736352,0.1,3.0,1013.0,1014.060578
2022-05-17 13:00:00,948.337344,0.1,3.0,1013.0,1013.741071
2022-05-17 14:00:00,905.315856,0.1,3.0,1013.0,1012.782859


In [47]:
# convert to xarray and confirm whether coordinates are shared
met_hourly_ds = met_hourly_df.to_xarray()
print(met_hourly_ds.nbytes)
met_hourly_ds

8064


<xarray.Dataset> Size: 8kB
Dimensions:                     (time_h: 168)
Coordinates:
  * time_h                      (time_h) datetime64[ns] 1kB 2022-05-10T17:00:...
Data variables:
    q_Solar                     (time_h) float64 1kB 447.1 239.1 ... 802.7 649.0
    cloudiness                  (time_h) float64 1kB 0.1 0.1 0.1 ... 0.1 0.1 0.1
    wind_speed                  (time_h) float64 1kB 3.0 3.0 3.0 ... 3.0 3.0 3.0
    atmospheric_pressure        (time_h) float64 1kB 1.013e+03 ... 1.013e+03
    atmospheric_vapor_pressure  (time_h) float64 1kB 1.013e+03 ... 1.011e+03

### Check sizes

In [58]:
met_hourly_ds.time_h.nbytes

1344

In [59]:
met_hourly_ds.q_Solar.nbytes

1344

In [65]:
met_hourly_ds.q_Solar.values.nbytes

1344

In [64]:
# Confirm that size of dataset is equal to the size of each of the data arrays, only counting the coordinate once
1344 * (1+5)

8064

### Check Memory addresses

In [56]:
id(met_hourly_ds.time_h)

4982576800

In [55]:
id(met_hourly_ds.q_Solar.time_h)

4980861520

In [57]:
id(met_hourly_ds.cloudiness.time_h)

4980857824

NOTE: It looks like there is a different memory addres for the time coordiate when they are accessed via an xr.DataArray

## Convert to Xarray Datasets & Merge

In [28]:
df_list = [air_temperature_df, met_hourly_df]

In [29]:
met_bc_ds = xr.merge(
    [df.to_xarray() for df in df_list], 
)
met_bc_ds

<xarray.Dataset> Size: 31kB
Dimensions:                     (time_5min: 1440, time_h: 168)
Coordinates:
  * time_5min                   (time_5min) datetime64[ns] 12kB 2022-05-12 .....
  * time_h                      (time_h) datetime64[ns] 1kB 2022-05-10T17:00:...
Data variables:
    TairC                       (time_5min) float64 12kB 66.0 66.0 ... 72.0 72.0
    q_Solar                     (time_h) float64 1kB 447.1 239.1 ... 802.7 649.0
    cloudiness                  (time_h) float64 1kB 0.1 0.1 0.1 ... 0.1 0.1 0.1
    wind_speed                  (time_h) float64 1kB 3.0 3.0 3.0 ... 3.0 3.0 3.0
    atmospheric_pressure        (time_h) float64 1kB 1.013e+03 ... 1.013e+03
    atmospheric_vapor_pressure  (time_h) float64 1kB 1.013e+03 ... 1.011e+03

In [ ]:
met_bc_ds.nbytes

31104

In [33]:
met_bc_ds.time_5min.nbytes

11520

In [44]:
met_bc_ds.time_h.values.nbytes

1344

# Demo Multiple Time Coordinates in a Merged Dataset
and loop through the different variables

## Load Xarray Tutorial Datasets

In [3]:
input_ds = xr.tutorial.open_dataset('air_temperature')
input_ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Create new DataArray with downsampled time dimension

In [4]:
# Create new DataArray with downsampled time dimension
daily_da = input_ds.air.resample(time='1D').mean() - 273.15
daily_da = daily_da.rename({'time': 'day'})
daily_da.name = 'temp'
daily_da.attrs['units'] = 'degC'
daily_da

<xarray.DataArray 'temp' (day: 730, lat: 25, lon: 53)> Size: 8MB
array([[[-31.2775, -30.85  , -30.475 , ..., -39.7775, -37.975 ,
         -35.475 ],
        [-28.575 , -28.5775, -28.875 , ..., -41.9025, -40.325 ,
         -36.85  ],
        [-19.15  , -19.9275, -21.3275, ..., -41.675 , -39.455 ,
         -34.525 ],
        ...,
        [ 23.15  ,  22.825 ,  22.85  , ...,  22.7475,  22.17  ,
          21.795 ],
        [ 23.175 ,  23.575 ,  23.5925, ...,  23.0225,  22.85  ,
          22.3975],
        [ 23.47  ,  23.845 ,  23.95  , ...,  23.8725,  23.8975,
          23.8225]],

       [[-29.55  , -29.65  , -29.85  , ..., -34.1775, -32.3525,
         -30.0775],
        [-25.3275, -25.95  , -26.9275, ..., -37.225 , -36.5525,
         -34.55  ],
        [-19.6275, -21.0775, -22.8525, ..., -35.4525, -34.2775,
         -31.25  ],
...
        [ 23.215 ,  22.265 ,  22.015 , ...,  23.74  ,  23.195 ,
          22.195 ],
        [ 24.3675,  24.515 ,  23.895 , ...,  23.415 ,  22.995 ,
          22.27  ],
        [ 25.4175,  25.5925,  25.1925, ...,  23.6425,  23.19  ,
          22.72  ]],

       [[-28.935 , -29.535 , -30.385 , ..., -29.41  , -28.96  ,
         -28.46  ],
        [-23.835 , -24.06  , -24.56  , ..., -32.585 , -31.635 ,
         -30.035 ],
        [-10.21  , -10.785 , -11.435 , ..., -33.685 , -31.035 ,
         -27.135 ],
        ...,
        [ 21.69  ,  21.99  ,  23.49  , ...,  22.265 ,  22.015 ,
          21.415 ],
        [ 23.39  ,  24.44  ,  24.94  , ...,  22.415 ,  22.315 ,
          21.64  ],
        [ 24.84  ,  25.59  ,  25.54  , ...,  23.065 ,  22.715 ,
          22.39  ]]], shape=(730, 25, 53))
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * day      (day) datetime64[ns] 6kB 2013-01-01 2013-01-02 ... 2014-12-31
Attributes:
    units:    degC

In [5]:
# add daily dataarray to input dataset
input_ds['temp'] = daily_da
input_ds

<xarray.Dataset> Size: 39MB
Dimensions:  (lat: 25, time: 2920, lon: 53, day: 730)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
  * day      (day) datetime64[ns] 6kB 2013-01-01 2013-01-02 ... 2014-12-31
Data variables:
    air      (time, lat, lon) float64 31MB ...
    temp     (day, lat, lon) float64 8MB -31.28 -30.85 -30.47 ... 22.72 22.39
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Run a Process on along two time coordinates

In [10]:
for step in input_ds.time.values:
    if step in input_ds.day.values:
        print(step)

2013-01-01T00:00:00.000000000
2013-01-02T00:00:00.000000000
2013-01-03T00:00:00.000000000
2013-01-04T00:00:00.000000000
2013-01-05T00:00:00.000000000
2013-01-06T00:00:00.000000000
2013-01-07T00:00:00.000000000
2013-01-08T00:00:00.000000000
2013-01-09T00:00:00.000000000
2013-01-10T00:00:00.000000000
2013-01-11T00:00:00.000000000
2013-01-12T00:00:00.000000000
2013-01-13T00:00:00.000000000
2013-01-14T00:00:00.000000000
2013-01-15T00:00:00.000000000
2013-01-16T00:00:00.000000000
2013-01-17T00:00:00.000000000
2013-01-18T00:00:00.000000000
2013-01-19T00:00:00.000000000
2013-01-20T00:00:00.000000000
2013-01-21T00:00:00.000000000
2013-01-22T00:00:00.000000000
2013-01-23T00:00:00.000000000
2013-01-24T00:00:00.000000000
2013-01-25T00:00:00.000000000
2013-01-26T00:00:00.000000000
2013-01-27T00:00:00.000000000
2013-01-28T00:00:00.000000000
2013-01-29T00:00:00.000000000
2013-01-30T00:00:00.000000000
2013-01-31T00:00:00.000000000
2013-02-01T00:00:00.000000000
2013-02-02T00:00:00.000000000
2013-02-03